# Law Preprocessing

In [1]:
import json
import glob
import natsort
from tqdm import tqdm

In [2]:
# ./law_text 디렉토리에 있는 모든 json 파일을 읽어서 하나의 데이터프레임으로 만들기
law_text_list = []

for i in natsort.natsorted(glob.glob('../../data/law_text/*.json')):
    with open(i, 'r') as f:
        law_text = json.load(f)
        law_text_list.append(law_text)

In [3]:
import re

def replace_multiple_spaces(text):
    # 연속된 공백을 하나의 공백으로 치환
    return re.sub(r'\s+', ' ', text)

def recursive_replace(data):
    if isinstance(data, dict):
        # 딕셔너리의 경우: 각 키와 값에 대해 재귀적으로 함수를 적용
        return {key: recursive_replace(value) for key, value in data.items()}
    elif isinstance(data, list):
        # 리스트의 경우: 각 요소에 대해 재귀적으로 함수를 적용
        return [recursive_replace(item) for item in data]
    elif isinstance(data, str):
        # 문자열의 경우: 공백 치환 수행
        return replace_multiple_spaces(data)
    return data  # 그 외 데이터 타입의 경우: 변경 없음

In [5]:
# 예제 딕셔너리
example_dict = {
    "key1": "여기\t\t\t    많은\t\t     공백이  있어요.",
    "key2": {"subkey1": [{"많은\t\t     공백": "많은\t\t     공백이  있어요."}, "또\t\t다른    문자열\t입니다.", "또다른      문자열\t2\n입니다."]},
    "key3": "이것은\t\t변경될\t\t\t문자열입니다."
}

# 함수 호출
result_dict = recursive_replace(example_dict)
print(result_dict)

{'key1': '여기 많은 공백이 있어요.', 'key2': {'subkey1': [{'많은\t\t     공백': '많은 공백이 있어요.'}, '또 다른 문자열 입니다.', '또다른 문자열 2 입니다.']}, 'key3': '이것은 변경될 문자열입니다.'}


In [7]:
law_text_cleansed = []

for i in tqdm(range(len(law_text_list))):
    law_text_cleansed.append(recursive_replace(law_text_list[i]))

100%|██████████| 5311/5311 [00:06<00:00, 872.15it/s] 


In [12]:
law_text_cleansed[3]

{'법령ID': '000004',
 '공포일자': '20230304',
 '공포번호': '19228',
 '언어': '한글',
 '법종구분': '법률',
 '법령명_한글': '재외공관 공증법',
 '법령명_한자': '재외공관 공증법',
 '법령명약칭': None,
 '제명변경여부': 'N',
 '한글법령여부': 'N',
 '편장절관': '44030000',
 '소관부처': '외교부',
 '전화번호': '02-2100-7568',
 '시행일자': '20230605',
 '제개정구분': '타법개정',
 '별표편집여부': 'N',
 '공포법령여부': 'Y',
 '공동부령정보': None,
 '조문': [{'조문번호': '1',
   '조문여부': '전문',
   '조문제목': None,
   '조문시행일자': '20230605',
   '조문제개정유형': None,
   '조문이동이전': None,
   '조문이동이후': None,
   '조문변경여부': 'N',
   '조문내용': ' 제1장 총칙 <개정 2009.12.30> '},
  {'조문번호': '1',
   '조문여부': '조문',
   '조문제목': '적용',
   '조문시행일자': '20230605',
   '조문제개정유형': None,
   '조문이동이전': None,
   '조문이동이후': None,
   '조문변경여부': 'N',
   '조문내용': '제1조(적용) 대한민국 영토 밖에서의 공증(公證)에 관한 사무는 이 법에서 정하는 바에 따른다. '},
  {'조문번호': '2',
   '조문여부': '조문',
   '조문제목': '공증사무의 담당',
   '조문시행일자': '20230605',
   '조문제개정유형': None,
   '조문이동이전': None,
   '조문이동이후': None,
   '조문변경여부': 'N',
   '조문내용': '제2조(공증사무의 담당) ',
   '항': [{'항번호': '①',
     '항제개정유형': None,
     '항내용': ' ① 제1조에 따른

In [13]:
for i in tqdm(range(len(law_text_cleansed))):
    with open(f'../../data/panre/law_text_cleansed/{law_text_cleansed[i]["법령ID"]}.json', 'w') as f:
        json.dump(law_text_cleansed[i], f, ensure_ascii=False, indent=4)

100%|██████████| 5311/5311 [00:06<00:00, 877.87it/s] 


----